# Solving Sudoku

I'm going to attempt to solve Sudoku puzzles using 2 methods:

* Linear Algebra
    - When I first saw Sudoku years ago, the first thing I thought of was a system of linear equations - the puzzle is a matrix after all. So I want to explore the idea of writing a system of linear equations to describe a Sudoku puzzle, and use some numeric optimization algorithms to try to find a solution (by minimizing the "error").
* Backtrack searching
    - This is the most obvious and probably the best way to solve Sudoku
    
In both cases, we'll develop a solution on a 4x4 version of sudoku before moving to the normal 9x9 board.

## Solving Sudoku with Linear Algebra

### Pretty-print matrix function

In [159]:
from IPython.display import HTML

def print_matrix(mat):
    t = ["<table>"]
    for row in mat:
        t.append("<tr>")
        for col in row:
            t.append("<td>" + str(col) + "</td>")
        t.append("</tr>")
    t.append("</table>")
    return HTML(''.join(t))

print_matrix([[1,2],[3,4]])

1,2
3,4


In [160]:
import numpy as np
import random

## 4x4 Sudoku

<table class="invisible-table"><tr><td>

Puzzle
<table class="padded-table">
<tr><td><b>1</b></td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
<tr><td> </td><td>&nbsp;</td><td><b>4</b></td><td>&nbsp;</td></tr>
<tr><td> </td><td><b>3</b></td><td>&nbsp;</td><td>&nbsp;</td></tr>
<tr><td>&nbsp;</td><td>&nbsp;</td><td><b>2</b></td><td>&nbsp;</td></tr>
</table>

</td><td>

Solution
<table class="padded-table">
<tr><td>1</td><td>4</td><td>3</td><td>2</td></tr>
<tr><td>3</td><td>2</td><td>4</td><td>1</td></tr>
<tr><td>2</td><td>3</td><td>1</td><td>4</td></tr>
<tr><td>4</td><td>1</td><td>2</td><td>3</td></tr>
</table>

</td></tr></table>

<style>
.padded-table td { padding: 15px; }
.invisible-table { border: 0px; }
</style>


In [161]:
m1 = [['a01', 'a02', 'a03', 'a04'],
      ['a05', 'a06', 'a07', 'a08'],
      ['a09', 'a10', 'a11', 'a12'],
      ['a13', 'a14', 'a15', 'a16']]

print_matrix(m1)

a01,a02,a03,a04
a05,a06,a07,a08
a09,a10,a11,a12
a13,a14,a15,a16


### System of Equations to describe puzzle

#### Sudoku-constant equations

These first 12 equations are the same for all 4x4 sudoku puzzles - they are the rules of the game:

By Rows:

* a01 + a02 + a03 + a04 = 10
* a05 + a06 + a07 + a08 = 10
* a09 + a10 + a11 + a12 = 10
* a13 + a14 + a15 + a16 = 10

By Columns:

* a01 + a05 + a09 + a13 = 10
* a02 + a06 + a10 + a14 = 10
* a03 + a07 + a11 + a15 = 10
* a04 + a08 + a12 + a16 = 10

By Quandrants:

* a01 + a02 + a05 + a06 = 10
* a03 + a04 + a07 + a08 = 10
* a09 + a10 + a13 + a14 = 10
* a11 + a12 + a15 + a16 = 10


#### Puzzle-specific equations

These last 4 equations are what we are given for this particular puzzle:

* a01 = 1
* a07 = 4
* a10 = 3
* a15 = 2



In [162]:
a =   [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

b = [10, 10, 10, 10,  10, 10, 10, 10,  10, 10, 10, 10,  1, 4, 3, 2]

In [163]:
x = np.linalg.lstsq(a,b)
np.reshape(x[0], (4,-1))

array([[ 1.        ,  3.47387651,  0.66316228,  4.86296121],
       [ 3.92477232,  1.60135117,  4.        ,  0.47387651],
       [ 2.04450137,  3.        ,  3.33683772,  1.61866091],
       [ 3.03072631,  1.92477232,  2.        ,  3.04450137]])

In [164]:
solution = np.reshape([int(round(i)) for i in x[0]], (4, -1))
solution

array([[1, 3, 1, 5],
       [4, 2, 4, 0],
       [2, 3, 3, 2],
       [3, 2, 2, 3]])

# Linear Regression Implementation

I wanted to implement my own linear regression algorithm to better understand how it works.

In [165]:
# TODO: Add precesion early break
def linear_regression(a, b, precision=0.01, learning_rate=0.1, max_iters = 200):
    m = len(b)
    omega = np.ones(m)
    
    for iter in range(max_iters):
        for i in range(m):
            omega[i] = omega[i] - learning_rate * (1.0/m) * sum((np.dot(a, omega) - b) * a[i])
        
    return omega

### Test problem to solve

In [166]:
# 3x+4y=7 and 5x+6y=8
a1=[[3,4],[5,6]]
b1=[7,8]
linear_regression(a1,b1)

array([-5. ,  5.5])

### Compare with numpy implementation's solution

In [167]:
((x,y),_,_,_) = np.linalg.lstsq(a1,b1)
(x,y)

(-4.9999999999999769, 5.4999999999999813)

In [168]:
result = linear_regression(a,b)
result

array([  4.71143849,   4.688242  ,   4.66519049,  -0.71713789,
         1.9568043 ,  -6.86351593,   1.06255254,  17.08008666,
        -1.34069241,  10.54590296,  -3.6327123 ,   7.43473406,
        -0.87012213,   7.11479335,  -4.74101031,  -2.80275689])

# Trying on Normal (9x9) Sudoku Puzzle

In [169]:
s9x9_flat = np.array(['a'+str(i) for i in range(81)])
s9x9 = np.reshape(s9x9_flat, (9, -1))

print_matrix(s9x9)

a0,a1,a2,a3,a4,a5,a6,a7,a8
a9,a10,a11,a12,a13,a14,a15,a16,a17
a18,a19,a20,a21,a22,a23,a24,a25,a26
a27,a28,a29,a30,a31,a32,a33,a34,a35
a36,a37,a38,a39,a40,a41,a42,a43,a44
a45,a46,a47,a48,a49,a50,a51,a52,a53
a54,a55,a56,a57,a58,a59,a60,a61,a62
a63,a64,a65,a66,a67,a68,a69,a70,a71
a72,a73,a74,a75,a76,a77,a78,a79,a80


### Generate equations (in matrix form)

In [170]:
sudoku_base = []
num_rows = 9
num_cols = 9

# Generate equations for rows
for row in range(num_rows):
    row_offset = num_rows * row
    new_row = [0 for i in range(num_rows * num_cols)]
    for i in range(num_rows):
        new_row[row_offset + i] = 1
    sudoku_base.append(new_row)

# Generate equations for columns
for col in range(num_cols):
    col_offset = num_cols * col
    new_row = [0 for i in range(num_rows * num_cols)]
    for i in range(num_cols):
        new_row[i * 9 + col] = 1
    sudoku_base.append(new_row)

# Generate equations for squares
current_cell = 0
for square_start in [0, 3, 6, 27, 30, 33, 54, 57, 60]:
    new_row = [0 for i in range(num_rows * num_cols)]
    for cell_offset in [0, 1, 2, 9, 10, 11, 18, 19, 20]:
        new_row[square_start+cell_offset] = 1
    sudoku_base.append(new_row)

In [171]:
# This is a wide matrix
#print_matrix(sudoku_base)

In [172]:
# Definition of sample puzzle
puzzle1 = {0:8, 3:1, 4:6, 9:4, 10:1, 
          12:9, 14:8, 17:6, 25:1, 31:8, 
          32:6, 35:3, 37:8, 38:9, 40:3, 
          42:2, 43:5, 45:3, 48:2, 49:9, 
          55:2, 63:7, 66:8, 68:4, 70:6, 
          71:5, 76:5, 77:7, 80:4}

In [173]:
def print_puzzle(puzzle):
    puzzle_mat = [' ' for i in range(81)]
    for (k,v) in puzzle.items():
        puzzle_mat[k] = v
    return print_matrix(np.reshape(puzzle_mat, (9, -1)))

In [174]:
print_puzzle(puzzle1)

8,,,1,6,,,,
4,1,,9,,8,,,6
,,,,,,,1,
,,,,8,6,,,3
,8,9,,3,,2,5,
3,,,2,9,,,,
,2,,,,,,,
7,,,8,,4,,6,5
,,,,5,7,,,4


In [175]:
print("Number of equations:", len(puzzle1)+len(a))

Number of equations: 45


In [176]:
print("Number of unknowns:", 81 - len(puzzle1))

Number of unknowns: 52


In [177]:
def get_puzzle_rep(puzzle):
    a = sudoku_base[:]
    b = [sum(range(1,10)) for i in range(len(a))]
    for (k,v) in puzzle.items():
        new_a_row = [0 for i in range(81)]
        new_a_row[k] = 1
        a.append(new_a_row)
        b.append(v)
    return (a,b)


In [178]:
p1_a, p1_b = get_puzzle_rep(puzzle1)

In [179]:
len(p1_a)

56

In [180]:
x = np.linalg.lstsq(p1_a,p1_b)
solution = np.reshape([int(round(i)) for i in x[0]], (9, -1))
solution

array([[  8,  75, -29,   1,   6,  -1, -10,  11, -15],
       [  4,   1,  11,   9,   3,   8,   3,   0,   6],
       [  6, -29,  -1,  17,   2,   0,  22,   1,  29],
       [ -8,   1,  11,   6,   8,   6,  10,   9,   3],
       [  8,   8,   9,  -6,   3,  11,   2,   5,   5],
       [  3,   1,  14,   2,   9,   5,   1,   6,   4],
       [  7,   2,  14,   6,   2,   5,  -1,   7,   4],
       [  7, -11,  17,   8,   7,   4,   2,   6,   5],
       [ 12,  -2,  -1,   2,   5,   7,  16,   1,   4]])

# Constrained Linear Regression - scipy.optimize.nnls

In [181]:
import scipy.optimize

In [182]:
result = scipy.optimize.nnls(p1_a, p1_b)

In [183]:
solution = np.reshape([int(round(i)) for i in result[0]], (9, -1))
solution

array([[ 8, 30,  0,  1,  6,  0,  0,  0,  0],
       [ 4,  1,  0,  9,  0,  8, 17,  0,  6],
       [ 0,  2,  0,  0,  1, 20, 21,  1,  0],
       [23,  0,  0,  0,  8,  6,  5,  0,  3],
       [ 0,  8,  9, 17,  3,  0,  2,  5,  1],
       [ 3,  2,  0,  2,  9,  0,  0,  3, 26],
       [ 0,  2, 21,  8, 13,  0,  0,  1,  0],
       [ 7,  0, 15,  8,  0,  4,  0,  6,  5],
       [ 0,  0,  0,  0,  5,  7,  0, 29,  4]])

# scipy.optimize.fmin_slsqp


In [184]:
import scipy.optimize

In [185]:
def objective_func(x):
    mat_result = sum((np.dot(p1_a, x) - p1_b)**2)
    #duplicate_count = 2 * sum([ (9 - len(set(x[row_start:row_start+9]))) for row_start in xrange(0, 81, 9)])
    return mat_result # + duplicate_count

In [186]:
objective_func(result[0])

4.833449373902292e-27

In [187]:
random_guess = [random.randint(1, 9) for i in range(81)]
objective_func(random_guess)

1672

In [188]:
slsqp_result = scipy.optimize.fmin_slsqp(func=objective_func, x0=random_guess, bounds=[(1,9) for i in range(81)])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2.29797100389e-07
            Iterations: 32
            Function evaluations: 2686
            Gradient evaluations: 32


In [189]:
slsqp_result

array([ 8.00003261,  6.35940875,  4.93648889,  1.00000169,  5.99997624,
        4.29737636,  7.04763379,  3.92579154,  3.43323742,  3.99996711,
        1.00004054,  2.86312341,  9.        ,  2.67226732,  8.00000642,
        5.71913519,  5.74536874,  6.00004407,  5.37036786,  7.29314808,
        5.17739899,  6.17573459,  5.39561016,  2.45905979,  5.88239592,
        1.        ,  6.24634972,  3.72172385,  7.22664993,  4.04442496,
        3.75455248,  7.99992386,  6.00006481,  3.23362232,  6.0189181 ,
        3.00007657,  3.1967975 ,  7.99994184,  9.        ,  5.28159176,
        3.00001894,  4.88903597,  1.99993536,  4.9999054 ,  4.63270593,
        3.0000033 ,  3.1900495 ,  3.62047293,  1.99994369,  9.        ,
        3.07474676,  7.94825758,  6.5796729 ,  6.58696652,  7.07995089,
        1.99997988,  5.20345885,  5.29346834,  2.51483276,  5.27972021,
        5.92777755,  5.60008543,  6.10076261,  6.99997325,  4.10688862,
        3.74539955,  8.00006622,  3.41739541,  4.00005442,  3.73

In [190]:
z = np.reshape([int(round(i)) for i in slsqp_result], (9, -1))
z

array([[8, 6, 5, 1, 6, 4, 7, 4, 3],
       [4, 1, 3, 9, 3, 8, 6, 6, 6],
       [5, 7, 5, 6, 5, 2, 6, 1, 6],
       [4, 7, 4, 4, 8, 6, 3, 6, 3],
       [3, 8, 9, 5, 3, 5, 2, 5, 5],
       [3, 3, 4, 2, 9, 3, 8, 7, 7],
       [7, 2, 5, 5, 3, 5, 6, 6, 6],
       [7, 4, 4, 8, 3, 4, 4, 6, 5],
       [4, 6, 6, 4, 5, 7, 4, 5, 4]])

In [191]:
objective_func(slsqp_result)

2.2979710038864857e-07

# Genetic Algorithm

The fmin_slsqp result seemed so close, I was curious if running the GA on its output would tweak the last few values into place.  It turns out not to work.

In [192]:
def sudoku_fitness(a, b, attempt):
    return sum((np.dot(a, attempt) - b)**2)

def sudoku_mutation_swap(current, max_mutations=3):
    num_mutations = random.randint(1, max_mutations)
    baby = current[:]
    for i in range(num_mutations):
        index1 = random.randint(0,80)
        index2 = random.randint(0,80)
        baby[index1], baby[index2] = baby[index2], baby[index1]
    return baby

def sudoku_mutation_tweak(current, max_mutations=3):
    num_mutations = random.randint(1, max_mutations)
    baby = current[:]
    for i in range(max_mutations):
        index1 = random.randint(0,80)
        index2 = random.randint(0,80)
        tweak_amount = random.randint(1, 3)
        baby[index1] = ((baby[index1]+tweak_amount) % 9) + 1
        baby[index2] = ((baby[index2]+tweak_amount) % 9) + 1
    return baby

def sudoku_genetic_algorithm(fitness_func, mutation_func, initial_guess=None, max_iters=1000):
    iter_count = 0
    error_steps = []
    current = np.array([i for i in range(1,10) for j in range(1,10)])
    current_fitness = 100000000 # really big - assume the guess is really bad

    if initial_guess != None:
        current = initial_guess[:]
        current_fitness = fitness_func(current)
    
    for i in range(max_iters):
        baby = mutation_func(current)
        baby_fitness = fitness_func(baby)
        
        # If we found a perfect solution, just return it!
        if baby_fitness == 0:
            return (baby, 0)
        
        # Compare baby fitness to current
        elif baby_fitness < current_fitness:
            print("Doing switch: baby_fitness=%d, current_fitness=%d" % (baby_fitness, current_fitness))
            current = baby[:]
            current_fitness = baby_fitness
        
        iter_count += 1
        if iter_count % 10000 == 0:
            print("Error (steps=%d): %d" % (iter_count, current_fitness))
            error_steps.append(current_fitness)
            
    return (current, error_steps)

In [193]:
initial_g = np.array([int(round(i)) for i in slsqp_result])
fitness = lambda attempt: sudoku_fitness(p1_a, p1_b, attempt)
print("Initial fitness:", fitness(initial_g))
(ga_result, ga_error) = sudoku_genetic_algorithm(fitness, sudoku_mutation_tweak, initial_guess=initial_g, max_iters=50000)


Initial fitness: 17


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:30: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Error (steps=10000): 17
Error (steps=20000): 17
Error (steps=30000): 17
Error (steps=40000): 17
Error (steps=50000): 17


In [194]:
np.reshape([int(round(i)) for i in ga_result], (9, -1))

array([[8, 6, 5, 2, 8, 8, 6, 7, 7],
       [3, 8, 5, 2, 6, 2, 2, 3, 8],
       [7, 3, 9, 2, 4, 1, 5, 3, 2],
       [4, 3, 7, 1, 2, 2, 3, 2, 4],
       [5, 3, 7, 3, 7, 6, 3, 1, 1],
       [4, 3, 1, 5, 8, 5, 5, 8, 9],
       [5, 4, 7, 6, 6, 5, 7, 5, 2],
       [8, 2, 3, 1, 1, 5, 7, 8, 9],
       [7, 6, 9, 8, 9, 4, 9, 2, 6]])

In [195]:
np.reshape([int(round(i)) for i in initial_g], (9, -1))

array([[8, 6, 5, 2, 8, 8, 6, 7, 7],
       [3, 8, 5, 2, 6, 2, 2, 3, 8],
       [7, 3, 9, 2, 4, 1, 5, 3, 2],
       [4, 3, 7, 1, 2, 2, 3, 2, 4],
       [5, 3, 7, 3, 7, 6, 3, 1, 1],
       [4, 3, 1, 5, 8, 5, 5, 8, 9],
       [5, 4, 7, 6, 6, 5, 7, 5, 2],
       [8, 2, 3, 1, 1, 5, 7, 8, 9],
       [7, 6, 9, 8, 9, 4, 9, 2, 6]])

# Backtracking

Okay, enough messing around; let's do it the right way now: backtracking.

### Define the representation

In [196]:
N = None
puzzle = [[1, N, N, N],
          [N, N, 4, N],
          [N, 3, N, N],
          [N, N, 2, N]]

### has_duplicates

This function returns true if there are no duplicates in the list. We'll use it to determine if a possible solution is breaking any rules of sudoku - that there are duplicate numbers in either a row, column, or "section" (board quandrant for a 4x4 board).

In [197]:
def has_duplicates(lst):
    seen = set()
    for i in lst:
        if i is not None:
            if i in seen:
                return True
            else:
                seen.add(i)
    return False

In [198]:
has_duplicates([1,2,3,4,5])

False

In [199]:
has_duplicates([1,2,3,4,5,1])

True

### get_row

In [200]:
def get_row(board, index):
    return board[index]

In [201]:
get_row(puzzle, 0)

[1, None, None, None]

### get_column

In [202]:
def get_column(board, index):
    return [board[i][index] for i in range(len(board))]

In [203]:
get_column(puzzle, 0)

[1, None, None, None]

### get_section

In [204]:
import math

def get_section(board, index):
    sqrt = int(math.sqrt(len(board)))
    start_row = (index // sqrt) * sqrt
    start_col = (index % sqrt) * sqrt
    section = []
    for i in range(sqrt):
        section += board[start_row][start_col:start_col+sqrt]
        start_row += 1
    return section

In [205]:
get_section(puzzle, 3)

[None, None, 2, None]

### copy_and_set

Return a new board with the specified value set.

In [206]:
def copy_and_set(board, row, col, value):
    board_new = [row.copy() for row in board]
    board_new[row][col] = value
    return board_new

In [207]:
copy_and_set(puzzle, 0, 1, 1)

[[1, 1, None, None],
 [None, None, 4, None],
 [None, 3, None, None],
 [None, None, 2, None]]

### is_valid

In [208]:
def is_valid(board):
    for row_index in range(len(board)):
        row = get_row(board, row_index)
        if has_duplicates(row):
            return False
    for col_index in range(len(board)):
        col = get_column(board, col_index)
        if has_duplicates(col):
            return False
    for section_index in range(len(board)):
        section = get_section(board, section_index)
        if has_duplicates(section):
            return False
    return True

In [209]:
# This should be not be valid, since it would be 2 '1's in the first row
is_valid(copy_and_set(puzzle, 0, 1, 1))

False

In [210]:
# This should be valid, since there are no duplicates in any rows, columns, or sections
is_valid(copy_and_set(puzzle, 0, 1, 2))

True

In [211]:
# This should not be valid because there are duplicate '1's in the first column
is_valid(copy_and_set(puzzle, 2, 0, 1))

False

In [212]:
# This should not be valid because there are duplicate '1's in the first section
is_valid(copy_and_set(puzzle, 1, 1, 1))

False

### solve_sudoku

In [213]:
def solve_sudoku(board, index=0):
    row_index = index // len(board)
    col_index = index % len(board)
    
    if (index > len(board)**2 - 1) or \
       (index == len(board)**2 - 1 and board[row_index][col_index] is not None):
        if is_valid(board):
            print('Solution found!')
            return board
        else:
            # No solution to this puzzle
            print('No solution to puzzle')
            return None
        
    if board[row_index][col_index] != None:     
        # Meaning it's a hard-coded value in the puzzle
        return solve_sudoku(board, index+1)
    
    for num in range(1, len(board)+1):
        with_num_set = copy_and_set(board, row_index, col_index, num)
        if is_valid(with_num_set):
            solution_or_none = solve_sudoku(with_num_set, index+1)
            if solution_or_none is not None:
                return solution_or_none

    # No solution found on this path
    return None

In [214]:
solution = solve_sudoku(puzzle)
np.reshape(solution, (4, -1))

Solution found!


array([[1, 4, 3, 2],
       [3, 2, 4, 1],
       [2, 3, 1, 4],
       [4, 1, 2, 3]])

## Let's try the backtracking solution on a 9x9 now

In [215]:
p_9x9 = [[8, N, N, 1, 6, N, N, N, N],
         [4, 1, N, 9, N, 8, N, N, 6],
         [N, N, N, N, N, N, N, 1, N],
         [N, N, N, N, 8, 6, N, N, 3],
         [N, 8, 9, N, 3, N, 2, 5, N],
         [3, N, N, 2, 9, N, N, N, N],
         [N, 2, N, N, N, N, N, N, N],
         [7, N, N, 8, N, 4, N, 6, 5],
         [N, N, N, N, 5, 7, N, N, 4]]

In [216]:
solution = solve_sudoku(p_9x9)
np.reshape(solution, (9, -1))

Solution found!


array([[8, 3, 5, 1, 6, 2, 7, 4, 9],
       [4, 1, 2, 9, 7, 8, 5, 3, 6],
       [9, 7, 6, 5, 4, 3, 8, 1, 2],
       [2, 5, 1, 7, 8, 6, 4, 9, 3],
       [6, 8, 9, 4, 3, 1, 2, 5, 7],
       [3, 4, 7, 2, 9, 5, 6, 8, 1],
       [5, 2, 4, 6, 1, 9, 3, 7, 8],
       [7, 9, 3, 8, 2, 4, 1, 6, 5],
       [1, 6, 8, 3, 5, 7, 9, 2, 4]])

# Conclusion

So backtracking, unsurprisingly, works well. But still, it was cool to see what Sudoku looks like as a system of equations right? :)